In [228]:
import langgraph
from langgraph.checkpoint.memory import MemorySaver
from langchain.tools.tavily_search import TavilySearchResults
from typing_extensions import TypedDict,Optional
from typing import List
from langchain.output_parsers import StructuredOutputParser
from pydantic import BaseModel
from langgraph.graph import StateGraph,END
from langchain_openai import ChatOpenAI
import os 
from dotenv import load_dotenv
from langchain_core.messages import SystemMessage,HumanMessage,AIMessage,AnyMessage
from tavily import TavilyClient


In [260]:
memory=MemorySaver()

In [261]:
load_dotenv()
os.environ["OPENAI_API_KEY"]=os.getenv("OPEN_API_KEY4")
os.environ["TAVILY_API_KEY"]=os.getenv("TAVILY_API_KEY")

In [262]:
class AgentState(TypedDict):
    task:str
    plan:str
    draft:str
    content:List[str]
    critique:str
    revision_number:int
    max_revisions:int

In [263]:
model=ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

In [264]:
tavily_s = TavilyClient(api_key=os.environ["TAVILY_API_KEY"])

In [265]:
PLAN_PROMPT = """You are an expert writer tasked with writing a high level outline of an essay. \
Write such an outline for the user provided topic. Give an outline of the essay along with any relevant notes \
or instructions for the sections."""

In [266]:
WRITER_PROMPT = """You are an essay assistant tasked with writing excellent 5-paragraph essays.\
Generate the best essay possible for the user's request and the initial outline. \
If the user provides critique, respond with a revised version of your previous attempts. \
Utilize all the information below as needed: 

------

{content}"""

In [267]:
REFLECTION_PROMPT = """You are a teacher grading an essay submission. \
Generate critique and recommendations for the user's submission. \
Provide detailed recommendations, including requests for length, depth, style, etc."""

In [268]:
RESEARCH_PLAN_PROMPT = """You are a researcher charged with providing information that can \
be used when writing the following essay. Generate a list of search queries that will gather \
any relevant information. Only generate 3 queries max."""


In [269]:
RESEARCH_CRITIQUE_PROMPT = """You are a researcher charged with providing information that can \
be used when making any requested revisions (as outlined below). \
Generate a list of search queries that will gather any relevant information. Only generate 3 queries max."""


In [270]:
from langchain_core.pydantic_v1 import BaseModel

class Queries(BaseModel):
    queries: List[str]


In [271]:
from langchain.tools.tavily_search import TavilySearchResults
tavily=TavilySearchResults(max_results=2)

In [272]:
def plan_node(state:AgentState):
    messages=[
        SystemMessage(content=PLAN_PROMPT),
        HumanMessage(content=state["task"])
    ]
    responce=model.invoke(
        messages
    )
    return {"plan":responce.content}

In [273]:
def research_plan_node(state: AgentState):
    queries = model.with_structured_output(Queries).invoke([
        SystemMessage(content=RESEARCH_PLAN_PROMPT),
        HumanMessage(content=state['task'])
    ])
    content = state.get("content", [])
    for q in queries.queries:
        response = tavily_s.search(query=q, max_results=2)
        for r in response['results']:
            content.append(r['content'])
    return {"content": content}

In [274]:
def generation_node(state:AgentState):
    content=f"\n\n".join(state["content"] or [])
    user_content=HumanMessage(content=f"{state['task']}\n\n here is my plan {state['plan']}")
    message=[
        SystemMessage(content=WRITER_PROMPT.format(content=content)),
        user_content
    ]
    responce=model.invoke(message)
    return {
        "draft":responce.content,
        "revision_number":state.get("revision_number",1)+1
    }
    

In [275]:
def reflection_node(state: AgentState):
    messages = [
        SystemMessage(content=REFLECTION_PROMPT), 
        HumanMessage(content=state['draft'])
    ]
    response = model.invoke(messages)
    return {"critique": response.content}

In [276]:
def research_critique_node(state: AgentState):
    queries = model.with_structured_output(Queries).invoke([
        SystemMessage(content=RESEARCH_CRITIQUE_PROMPT),
        HumanMessage(content=state['critique'])
    ])
    content = state.get("content", [])
    for q in queries.queries:
        response = tavily_s.search(query=q, max_results=2)
        for r in response['results']:
            content.append(r['content'])
    return {"content": content}

In [277]:
def should_continue(state):
    if state["revision_number"] > state["max_revisions"]:
        return END
    return "reflect"

In [278]:
graph=StateGraph(AgentState)
graph.add_node("planner",plan_node)

In [279]:

graph.add_node("generate",generation_node)
graph.add_node("reflect",reflection_node)
graph.add_node("research_plan", research_plan_node)
graph.add_node("research_critique", research_critique_node)


In [280]:
graph.add_conditional_edges(
    "generate",
    should_continue,
    {END:END,"reflect":"reflect"}
)

In [281]:
graph.set_entry_point("planner")
graph.add_edge("planner", "research_plan")
graph.add_edge("research_plan", "generate")

graph.add_edge("reflect", "research_critique")
graph.add_edge("research_critique", "generate")

In [282]:
graph_2 = graph.compile(checkpointer=memory)

In [283]:
thread = {"configurable": {"thread_id": "1"}}
for s in graph_2.stream({
    'task': "what is the difference between langchain and langsmith",
    "max_revisions": 3,
    "revision_number": 1,
}, thread):
    print(s)

{'planner': {'plan': 'I. Introduction\n    A. Brief overview of Langchain and Langsmith\n    B. Thesis statement: Exploring the differences between Langchain and Langsmith\n\nII. Langchain\n    A. Definition and explanation\n    B. Key features and characteristics\n    C. Use cases and applications\n    D. Advantages and disadvantages\n\nIII. Langsmith\n    A. Definition and explanation\n    B. Key features and characteristics\n    C. Use cases and applications\n    D. Advantages and disadvantages\n\nIV. Comparison between Langchain and Langsmith\n    A. Technology stack\n    B. Scalability\n    C. Security\n    D. Performance\n    E. Adoption and popularity\n\nV. Conclusion\n    A. Recap of main differences between Langchain and Langsmith\n    B. Implications for the future of blockchain technology\n    C. Final thoughts and recommendations\n\nNotes:\n- Ensure a clear and concise explanation of both Langchain and Langsmith.\n- Provide specific examples of use cases for each technology